# Twitter sentiment analysis

In [ ]:
import pandas as pd

COLUMNS  = ['target', 'id', 'date', 'flag', 'user', 'text']

# Reading data with appropriate encoding
data = pd.read_csv('twitter_data.csv', encoding='iso8859_2', names=COLUMNS)

In [ ]:
# What the data looks like
data.head()

### Types and shape

In [ ]:
data.dtypes

In [ ]:
data.shape

### Target value

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.countplot(data['target'], order=data['target'].value_counts().index)
plt.show()

The variable is balanced.

In [ ]:
sns.countplot(data['flag'], order=data['flag'].value_counts().index)
plt.show()

In [ ]:
text, target = list(data['text']), list(data['target'])

In [ ]:
import re

pattern_nick = re.compile(r'@[^\s]+')
pattern_http = re.compile(r'http:\/\/[^ ]+')
pattern_www = re.compile(r'www\.[^ ]+')
pattern_number = re.compile(r'[0-9]+')
pattern_signs = re.compile(r'[%,.!?-]')
pattern_letters = re.compile(r'[^a-zA-Z]')

In [ ]:
from nltk.stem import WordNetLemmatizer 
import nltk

lemmatizer = WordNetLemmatizer() 

In [ ]:
from nltk.stem import PorterStemmer 

ps = PorterStemmer()

In [ ]:
from nltk.corpus import stopwords

print(stopwords.words('english'))

In [ ]:
tweets = []

for tweet in text:
    tweet = tweet.lower()
    tweet = re.sub(pattern_nick, 'nick', tweet)
    tweet = re.sub(pattern_http, 'www', tweet)
    tweet = re.sub(pattern_www, 'www', tweet)
    tweet = re.sub(pattern_number, '', tweet)
    tweet = re.sub(pattern_signs, '', tweet)
    
    tweet = tweet.split()
    lemmatizer_tweet = ''
    for word in tweet: 
        if word not in set(stopwords.words('english')):
            lemmatizer_word = lemmatizer.lemmatize(word)
            stem_word = ps.stem(lemmatizer_word)
            lemmatizer_tweet += stem_word + ' '
    tweets.append(lemmatizer_tweet)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets, target, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectoriser = TfidfVectorizer()
vectoriser.fit(X_train)

In [ ]:
X_train = vectoriser.transform(X_train)
X_test  = vectoriser.transform(X_test)

In [ ]:
from sklearn.naive_bayes import BernoulliNB

BNBmodel = BernoulliNB(alpha = 2)
BNBmodel.fit(X_train, y_train)

In [ ]:
y_pred = BNBmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
tekst = ['I hate monday', 'good']

tekst  = vectoriser.transform(tekst)
BNBmodel.predict(tekst)